# Exoplanet Detection Using Machine Learning

## Introduction

In many real world situations, you will have limited or no labeled data.  Can we still use machine learning?  Yes!  However, our approach to the problem will change slightly.  The most common approach to unsupervised machine learning problems is clustering.  In today's workshop we'll explore another powerful machine learning approach, autoencoders.  Autoencoders are used for a number of different data science applications.  They are popular for anomaly detection, compression, and dimensional reduction.  Using an autoencoder converts the problem from an unsupervised machine learning problem to a semi-supervised machine learning problem.  

This notebook will use time series data from the [Kepler Spacecraft](https://en.wikipedia.org/wiki/Kepler_space_telescope).  Kepler is a space telescope launched in 2009 and retired in 2018.  It was designed to survey a portion of Earth's region of the Milky Way to discover Earth-size exoplanets in or near habitable zones.  Kepler's photometer continually monitored the brightness of approximately 150,000 main sequence stars in a fixed field of view. The data was transmitted to Earth, and then analyzed to detect periodic dimming caused by exoplanets that cross in front of their host star. Only planets whose orbits are seen edge-on from Earth could be detected. In total, Kepler observed 530,506 stars and detected 2,662 planets.

Kepler's method of exploplanet detection is known as the Transit Method.  The telescope observes the star's brightness as a function of time.  As exoplanets pass in front of the star the brightness drops slightly and then returns to the previous values.  Below is an illustration of this method by Chris Shallue.  In fact, we are going to build off of the excellent work of Chris Shallue, where they used machine learning to classify planet candidates.  

<img src="./images/transit.gif" width="800" align="center"/>
<p style="text-align: center;">Exoplanet Transit Method [via Chris Shallue]</p>

## Objective

The Kepler spacecraft generated A LOT of data (over 650 GB of science data).  Suppose we wanted to use a computer to find planet candidates from this data, but we don't have a labeled dataset.  Perhaps we could pay graduate student to label 1000 time series sequences.  Can we use this data of planet candidates to find other planet candidates and to distinguish planet candidates from other spurious signals collected by Kepler.  This approach is similar to a traditional anomaly detection problem where we have example time series of normal machine operation.  We can use the knowledge of normal behavior to identify abnormal behavior.  Let's get started!

# Resources

* https://github.com/google-research/exoplanet-ml/tree/master/exoplanet-ml/astronet
* Shallue, C. J., & Vanderburg, A. (2018). Identifying Exoplanets with Deep Learning: A Five-planet Resonant Chain around Kepler-80 and an Eighth Planet around Kepler-90. The Astronomical Journal, 155(2), 94.
* https://exoplanets.nasa.gov/exoplanet-catalog/6128/kepler-90g/
* https://exoplanets.nasa.gov/exoplanet-catalog/6129/kepler-90h/

Please use the tensorflow2.3 CPU / python 3.7 kernel with this notebook

In [ ]:
%pip install matplotlib seaborn

In [ ]:
# IMPORTS
import tensorflow as tf
import glob
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.tensorflow import TensorFlow
from sklearn.metrics import recall_score, classification_report, auc, roc_curve, precision_recall_curve, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import boto3
import time 
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output

# Data

For this workshop we will use data that has been prepared, transformed, and cleaned.  We will use a 1000 time series signals of planet cadidates.  This could have been labeled using graduate students using our hypothetical example.  Each time series has been scaled and centered, and includes a 'global' 2001 timestamp series and a 'local' 201 timestamp series. 

## Download Dataset

In [ ]:
# Download compressed archive (~138MB).
!mkdir data 
!aws s3 cp s3://aws-machine-learning-immersion-day/kepler-tce-tfrecords-20180220.tar ./data/

In [ ]:
!mkdir data/processed
!tar -xvf ./data/kepler-tce-tfrecords-20180220.tar -C ./data/processed/ --no-same-owner

In [ ]:
files = glob.glob('./data/processed/tfrecord/train*')
files

## Configure SageMaker

In [ ]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket_name = sagemaker.Session().default_bucket()

prefix = 'remars.exoplanet'

Terminology:

* TCE = Threshold Crossing Events
* PC = Planet Candidate
* NTP = Astrophysical False Positive - not planet
* AFP = Nontransiting Phenomenon - not planet

The predictions can be PC, NTP, or AFP


In [ ]:
tfrecord_format = {
            "av_pred_class": tf.io.FixedLenFeature([], tf.string),
            "av_training_set": tf.io.FixedLenFeature([], tf.string),
            "global_view": tf.io.FixedLenFeature([2001], tf.float32),
            "local_view": tf.io.FixedLenFeature([201],tf.float32),
            "kepid": tf.io.FixedLenFeature([],tf.int64),
            "spline_bkspace": tf.io.FixedLenFeature([],tf.float32),
            "tce_depth": tf.io.FixedLenFeature([],tf.float32),
            "tce_duration": tf.io.FixedLenFeature([],tf.float32),
            "tce_impact": tf.io.FixedLenFeature([],tf.float32),
            "tce_max_mult_ev": tf.io.FixedLenFeature([],tf.float32),
            "tce_model_snr": tf.io.FixedLenFeature([],tf.float32),
            "tce_period": tf.io.FixedLenFeature([],tf.float32),
            "tce_plnt_num": tf.io.FixedLenFeature([],tf.int64),
            "tce_prad": tf.io.FixedLenFeature([],tf.float32),
            "tce_time0bk": tf.io.FixedLenFeature([],tf.float32)
        }

In [ ]:
# Read the data back out.
def decode_fn(record_bytes):
  return tf.io.parse_single_example(
      # Data
      record_bytes,
      # Schema
      tfrecord_format
  )

In [ ]:
%%time
c = []
pc_global = []
pc_local = []
non_pc_global = []
non_pc_local = []
for rec in tf.data.TFRecordDataset(files).map(decode_fn):
    c.append(rec['av_pred_class'].numpy().decode('utf-8'))
    if 'PC' in rec['av_pred_class'].numpy().decode('utf-8'):
        pc_global.append(rec["global_view"].numpy())
        pc_local.append(rec["local_view"].numpy())
    else:
        non_pc_global.append(rec["global_view"].numpy())
        non_pc_local.append(rec["local_view"].numpy())
        
    if (rec["kepid"].numpy() == 11442793 and
          rec["tce_plnt_num"].numpy() == 1):
        kepler = rec
    if 'NTP' in rec['av_pred_class'].numpy().decode('utf-8'):
        ntp = rec
    if 'AFP' in rec['av_pred_class'].numpy().decode('utf-8'):  
        afp = rec

In [ ]:
print(f"Total number of planet candidates {c.count('PC')}") 
print(f"Total number of non planet candidates {len(c) - c.count('PC')}")

## Visualization

Let's take a look at a single planet candidate.  Here is an example of [Kepler-90](https://exoplanets.nasa.gov/exoplanet-catalog/6128/kepler-90g/) This is a G-type star in the Constellation Draco and is 2840 light years from Earth. 

In [ ]:
# Plot the global and local views.
global_view = np.array(kepler["global_view"].numpy())
local_view = np.array(kepler["local_view"].numpy())
fig, axes = plt.subplots(1, 2, figsize=(20, 6))
axes[0].plot(global_view, ".")
axes[1].plot(local_view, ".")
axes[0].title.set_text('planet candidate - global view')
axes[1].title.set_text('planet candidate - local view')
plt.show()

In [ ]:
# Plot the global and local views.
global_view = np.array(ntp["global_view"].numpy())
local_view = np.array(ntp["local_view"].numpy())
fig, axes = plt.subplots(1, 2, figsize=(20, 6))
axes[0].plot(global_view, ".")
axes[1].plot(local_view, ".")
axes[0].title.set_text('non transiting phenomenon - global view')
axes[1].title.set_text('non transiting phenomenon - local view')
plt.show()

In [ ]:
# Plot the global and local views.
global_view = np.array(afp["global_view"].numpy())
local_view = np.array(afp["local_view"].numpy())
fig, axes = plt.subplots(1, 2, figsize=(20, 6))
axes[0].plot(global_view, ".")
axes[1].plot(local_view, ".")
axes[0].title.set_text('astrophysical false positive - global view')
axes[1].title.set_text('astrophysical false positive - local view')
plt.show()

## Train | Validation Split

In this example, let's assume that we have 12000 example time series and we want to find the planet candidates.  Let's also assume that we don't have labeled data.  How do we approach a problem like this?  Our grad students identified 1000 planet candidates.  We'll use this data to label the remaining dataset.  Or said, differently, we'll use this data to determine other planet candidates and non planet candidates.  We will concatenate the local and global views together into a single 2202 timestamp time series.

In [ ]:
pc_local_numpy = np.array(pc_local)
pc_global_numpy = np.array(pc_global)
temp = np.concatenate((pc_global_numpy,pc_local_numpy),axis=1)

In [ ]:
# get 1000 samples of planet candidates
temp, temp_ = train_test_split(
    temp, 
    train_size=1000, 
    random_state=4321, 
    shuffle=True)

In [ ]:
non_pc_local_numpy = np.array(non_pc_local)
non_pc_global_numpy = np.array(non_pc_global)
temp_non = np.concatenate((non_pc_global_numpy,non_pc_local_numpy),axis=1)

In [ ]:
labels = [1]*len(temp_)+[0]*len(temp_non)

In [ ]:
temp_ = np.concatenate((temp_,temp_non),axis=0)

In [ ]:
# split data into 80% training, 20% validation
train, validation = train_test_split(
    temp, 
    test_size=.2, 
    random_state=4321, 
    shuffle=True)

In [ ]:
print(f'Training dataset shape: {train.shape}')
print(f'Validation dataset shape: {validation.shape}')

In [ ]:
np.savez('./data/training', train=train)
np.savez('./data/validation', validation=validation)

Upload datasets to S3

In [ ]:
training_input_path   = sess.upload_data('data/training.npz', bucket=bucket_name, key_prefix=prefix+'/training')
validation_input_path   = sess.upload_data('data/validation.npz', bucket=bucket_name, key_prefix=prefix+'/validation')
print('Uploaded training data location: {}'.format(training_input_path))
print('Uploaded validation data location: {}'.format(validation_input_path))

# Model Training

## Autoencoder

![autoencoder](./images/Autoencoder_structure.png)

AutoEncoders are a special kind of neural network, where your input is 'x' and your output is 'x' as well. What this really means is that we are trying to learn a function, where the input and output are the same.  In a linear system an autoencoder would be the identity matrix, however, we are going to use a non-linear neural network to construct our autoencoder.  

The function f(x), that we are going to learn is approximately equal to x

Few things to note.

* We are reducing the number of nodes in the hidden layers, which will force the network to learn the features from the dataset. Intuition being that this "code" is a set of abstracted features which represents or creates a fingerprint for the training dataset.
* Since we are starting with the input 'x', reducing into a abstracted features and then reconstructing back the 'x' means we don't need a labeled dataset.  We are converting the problem from an usupervised problem to a supervised problem.  
* The "code" is intutively a representation of abstracted features.

For our explanet dataset, we are going a dataset of 1000 time series that have been identified as 'planet cadidates'.  We will train the autoencoder on this data.  Next, we'll used the trained network to analyze a larger dataset.  We will use the reconstruction error (Mean Squared Error - MSE) to find other planet crossing data.  During this process the network should try to learn a unique representation of what a planet candidate time series looks like.  

In [ ]:
output_location = f's3://{bucket_name}/{prefix}/output'

print('Training artifacts will be uploaded to: {}'.format(output_location))

In [ ]:
!pygmentize ./code/autoencoder.fc.py

In [ ]:
tf_estimator = TensorFlow(entry_point='./code/autoencoder.fc.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.m5.xlarge',
                          framework_version='2.2', 
                          py_version='py37',
                          script_mode=True,
#                           use_spot_instances=True,
#                           max_run=3600,
#                           max_wait=3600,
                          hyperparameters={
                              'epochs': 30,
                              'batch-size': 64}
                         )

In [ ]:
tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

In [ ]:
# plot validation and training progress
client = boto3.client('logs')
BASE_LOG_NAME = '/aws/sagemaker/TrainingJobs'

def plot_log(model):
    logs = client.describe_log_streams(logGroupName=BASE_LOG_NAME, logStreamNamePrefix=model._current_job_name)
    cw_log = client.get_log_events(logGroupName=BASE_LOG_NAME, logStreamName=logs['logStreams'][0]['logStreamName'])

    val = []
    train = []
    iteration = []
    count = 0
    for e in cw_log['events']:
        msg = e['message']
        if 's - loss:' in msg:
            msg = msg.split(' ')
            #print(msg)
            train.append(float(msg[5]))
            val.append(float(msg[8]))
            iteration.append(count)
            count+=1

    fig, ax = plt.subplots(figsize=(15,5))
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    train_plot,   = ax.plot(iteration,   train,   label='train')
    val_plot,   = ax.plot(iteration,   val,   label='validation')
    plt.legend(handles=[train_plot,val_plot])
    plt.grid()
    plt.show()



In [ ]:
plot_log(tf_estimator)

# Deploy

In [ ]:
%%time

tf_endpoint_name = 'remars-autoencoder-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

tf_predictor = tf_estimator.deploy(initial_instance_count=1,
                         instance_type='ml.m5.xlarge',      
                         endpoint_name=tf_endpoint_name)    

In [ ]:
## How do you connect to an already deployed endpoint
# end_point_name = 'ENDPOINT_NAME'
# tf_predictor = sagemaker.tensorflow.model.TensorFlowPredictor(end_point_name,sagemaker_session=sess)

# Evaluate

## single prediction

In [ ]:
validation[1,:]

In [ ]:
a = tf_predictor.predict(validation[1,:]);

In [ ]:
plt.plot(validation[1,:],'b*',a['predictions'][0],'r*')
plt.legend(('actual','prediction'))

## hold out dataset prediction

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = []
    for array in split_array:
        predictions.extend(tf_predictor.predict(array)['predictions'])

    return predictions

In [ ]:
%%time
train_pred = predict(train)
val_pred = predict(validation)
pred_ = predict(temp_)

In [ ]:
train_mse = np.mean(np.power(train - train_pred, 2), axis=1)
val_mse = np.mean(np.power(validation - val_pred, 2), axis=1)
mse_ = np.mean(np.power(temp_ - pred_, 2), axis=1)

In [ ]:
error_df_val = pd.DataFrame({'reconstruction_error': val_mse})
error_df_val.describe(percentiles=[.50,.90,.95,.99,.999,.9999])

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(111)
_ = ax.hist(val_mse, bins=100, range=(0,1),density=True,color='blue',edgecolor='black',alpha=0.6,label='validation')
_ = ax.hist(mse_, bins=100, range=(0,1),density=True,color='red',edgecolor='black',alpha=0.4,label='hold_out')
plt.legend()
plt.xlabel('reconstruction error, MSE')
plt.ylabel('normalized count')


We can see from the reconstruction mean squared error above the validation data and the hold out data are different.  While there are some examples in the hold out dataset that have a near zero mean squared error, there are also some that have much higher reconstruction error.  The hold out dataset with MSE near zero is likely planet candidates and the time series with MSE greater than 0.1 are likely the non planet candidates.  

In [ ]:
# apply threshold 
def to_labels(pos_probs, threshold):
    return (pos_probs <= threshold).astype('int')

In [ ]:
fpr, tpr, thresholds = roc_curve(labels,mse_,pos_label=0)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.001, 1])
plt.ylim([0, 1.001])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show();


In [ ]:
precision, recall, th = precision_recall_curve(labels, mse_, pos_label=0)
plt.plot(recall, precision, 'b', label='Precision-Recall curve')
plt.title('Recall vs Precision')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
plt.plot(th, precision[1:], 'b', label='Threshold-Precision curve')
plt.title('Precision for different threshold values')
plt.xlabel('Threshold')
plt.ylabel('Precision')
plt.xlim((0,.2))
plt.show()

In [ ]:
plt.plot(th, recall[1:], 'b', label='Threshold-Recall curve')
plt.title('Recall for different threshold values')
plt.xlabel('Threshold')
plt.ylabel('Recall')
plt.xlim((0,.2))
plt.show()

In [ ]:
# define thresholds
thresholds = np.arange(0, 0.2, 0.001)
# evaluate each threshold
scores = [f1_score(labels, to_labels(mse_, t)) for t in thresholds]
# get best threshold
ix = np.argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

In [ ]:
plt.plot(thresholds, scores, 'b', label='F1 curve')
plt.title('F1 for different threshold values')
plt.xlabel('Threshold')
plt.ylabel('F1')
#plt.xlim((0,.2))
plt.show()

In [ ]:
thresh = 0.1

In [ ]:
class_list = ['NOT PC', 'PC']
fig, ax = plt.subplots(figsize=(15,10))
cm = confusion_matrix(labels,to_labels(mse_,thresh))
normalized_cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(normalized_cm, ax=ax, annot=cm, fmt='',xticklabels=class_list,yticklabels=class_list)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

If we use a threshold of 0.1 we can try to seperate out additional timeseries that are not planet crosssing.  Again, recall that we trained the network on planet candidates and timeseries that have high reconstruction error exhibit different behavior than the planet candidates.  When we use a threshold of 0.1 we identify ~3000 timeseries that are NOT planet candidates.  Similarly we could look for timeseries that have near zero reconstruction error to identify series of additional planet candidates.  

In [ ]:
print("Classification Report: ")
print(classification_report(y_true=labels, y_pred=to_labels(mse_,thresh)))

# Conclusion and Call to Action

We've shown above how to train a fully connected feed forward autoencoder.  In this approach we concatenated the two time series.  We used the reconstruction error to seperate the planet candidates from non planet cadidates.  However, we can also use an autoencoder architecture with a convolution or recurrent approach.  Using this notebook as a starting point, compare and contrast the performance of the fully connected autoencoder with a convolution or recurrent approach.  

## Clean up

In [ ]:
tf_predictor.delete.endpoint()